# Análisis OMPA con Simulaciones Monte Carlo
### Trabajo de Fin de Grado - Aitana Tomás San Martín
#### Tutor: Leopoldo Pena 
#### Ciéncias del Mar UB                    


Este cuaderno implementa una herramienta computacional que combina el método de análisis multiparamétrico óptimo (OMPA) con simulaciones de Monte Carlo, con el fin de mejorar la estimación de mezclas de masas de agua y cuantificar la incertidumbre asociada.

Este enfoque se ha aplicado a los datos de la campaña oceanográfica **TRANSMOW**, centrada en el margen ibérico atlántico. La implementación del Análisis Multiparámétrico Óptimo (OMPA) utilizada en este trabajo se basa en los principios y metodologías detalladas en el libro *Modeling Methods for Marine Science* de Glover, D. M., Jenkins, W. J., & Doney, S. C. (2011), Cambridge University Press (https://doi.org/10.1017/CBO9780511975721). Esta obra proporciona el marco teórico detallado y las herramientas conceptuales fundamentales para la correcta aplicación del OMPA en el contexto de las ciencias marinas.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import scipy.optimize
import seaborn as sns
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets, preprocessing

In [ ]:
data_t = pd.read_csv('C:/Users/Aitana/Documents/Python Scripts/OMPA FINAL/OMPA_ODV/data_ompa')

In [ ]:
data = data_t[['Salinity', 'Temperature', 'Oxygen', 'spice0']].copy()
data.columns = ['Salinity', 'Potential Temperature', 'Oxygen', 'spiciness']
print(data)

In [ ]:
endmembers = {
    "ENACW16": {"Theta": 16.0, "S": 36.20},
    "ENACW12": {"Theta": 12.3, "S": 35.66},
    "SPMW8": {"Theta": 8.0, "S": 35.00},
    "SPMW7": {"Theta": 7.1, "S": 35.16},
    "IrSPMW": {"Theta": 7.0, "S": 35.08},
    "LSW": {"Theta": 3.0, "S": 34.86},
    "SAIW6": {"Theta": 4.5, "S": 34.80},
    "SAIW4": {"Theta": 4.5, "S": 34.80},
    "MW": {"Theta": 11.7, "S": 36.2},
    "ISOW": {"Theta": 2.7, "S": 34.98},
    "NEADWu": {"Theta": 2.5, "S": 34.90},
    "NEADWl": {"Theta": 1.98, "S": 34.895},
}

plt.figure(figsize=(10, 6))
plt.scatter(data['Salinity'], data['Potential Temperature'], c='lightblue', label='Muestras', alpha=0.6)
for name, props in endmembers.items():
    plt.scatter(props['S'], props['Theta'], label=name, edgecolor='black', s=100, marker='X')
    plt.text(props['S'] + 0.02, props['Theta'], name, fontsize=8)

plt.gca()
plt.xlabel('Salinidad (PSU)')
plt.ylabel('Temperatura Potencial (°C)')
plt.title('Diagrama T-S con Endmembers')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

plt.savefig('diagrama_ts.png', dpi=300)

plt.show()

In [ ]:
endmembers = {
    "ENACW16": {"Theta": 16.0, "S": 36.20},
    "ENACW12": {"Theta": 12.3, "S": 35.66},
    "LSW": {"Theta": 3.0, "S": 34.86},
    "MW": {"Theta": 11.7, "S": 36.2},
    "NEADWu": {"Theta": 2.5, "S": 34.90},
}

plt.figure(figsize=(10, 6))

plt.scatter(data['Salinity'], data['Potential Temperature'], c='lightblue', label='Muestras', alpha=0.6)

for name, props in endmembers.items():
    plt.scatter(props['S'], props['Theta'], label=name, edgecolor='black', s=100, marker='X')
    plt.text(props['S'] + 0.02, props['Theta'], name, fontsize=8)

plt.gca()
plt.xlabel('Salinidad (PSU)')
plt.ylabel('Temperatura Potencial (°C)')
plt.title('Diagrama T-S con Endmembers')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

plt.savefig('diagrama_ts_5.png', dpi=300)

plt.show()

In [ ]:
design_M = pd.read_csv('C:/Users/Aitana/Documents/Python Scripts/OMPA FINAL/OMPA_ODV/Matriz_media_end.csv')
design_M_desvest = pd.read_csv('C:/Users/Aitana/Documents/Python Scripts/OMPA FINAL/OMPA_ODV/Matriz_std_end.csv')

In [ ]:
salinidad = data['Salinity']
temperatura = data['Potential Temperature']
oxigeno = data['Oxygen']

end_members = {
    'ENACW16': [design_M.loc[0, 'ENACW16'], design_M.loc[1, 'ENACW16'], design_M.loc[2, 'ENACW16']],
    'ENACW12': [design_M.loc[0, 'ENACW12'], design_M.loc[1, 'ENACW12'], design_M.loc[2, 'ENACW12']],
    'MW': [design_M.loc[0, 'MW'], design_M.loc[1, 'MW'], design_M.loc[2, 'MW']],
    'LSW': [design_M.loc[0, 'LSW'], design_M.loc[1, 'LSW'], design_M.loc[2, 'LSW']],
    'NEADW': [design_M.loc[0, 'NEADW'], design_M.loc[1, 'NEADW'], design_M.loc[2, 'NEADW']]
}

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

colors = {
    'ENACW16': 'red',
    'ENACW12': 'blue',
    'MW': 'green',
    'LSW': 'orange',
    'NEADW': 'purple'
}

sc = ax.scatter(salinidad, oxigeno, temperatura, c=oxigeno, cmap='viridis', label='Datos', alpha=0.1)

for end_member, values in end_members.items():
    ax.scatter(values[0], values[2], values[1], label=end_member, s=100, c=colors[end_member], marker='o', alpha=1)

ax.set_xlabel('Salinidad (PSU)')
ax.set_ylabel('Oxígeno (mg/L)')
ax.set_zlabel('Temperatura (°C)')
plt.colorbar(sc)
ax.set_title('Relación entre Salinidad, Oxígeno y Temperatura con End-Members')
ax.legend()

plt.show()

salinidad = data['Salinity']
temperatura = data['Potential Temperature']
oxigeno = data['Oxygen']

end_members = {
    'ENACW16': [design_M.loc[0, 'ENACW16'], design_M.loc[1, 'ENACW16'], design_M.loc[2, 'ENACW16']],
    'ENACW12': [design_M.loc[0, 'ENACW12'], design_M.loc[1, 'ENACW12'], design_M.loc[2, 'ENACW12']],
    'MW': [design_M.loc[0, 'MW'], design_M.loc[1, 'MW'], design_M.loc[2, 'MW']],
    'LSW': [design_M.loc[0, 'LSW'], design_M.loc[1, 'LSW'], design_M.loc[2, 'LSW']],
    'NEADW': [design_M.loc[0, 'NEADW'], design_M.loc[1, 'NEADW'], design_M.loc[2, 'NEADW']]
}

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

colors = {
    'ENACW16': 'red',
    'ENACW12': 'blue',
    'MW': 'green',
    'LSW': 'orange',
    'NEADW': 'purple'
}

sc = ax.scatter(salinidad, temperatura, oxigeno, c=oxigeno, cmap='viridis', label='Datos')

for end_member, values in end_members.items():
    ax.scatter(values[0], values[1], values[2], label=end_member, s=100, c=colors[end_member], marker='o')

ax.set_xlabel('Salinidad (PSU)')
ax.set_ylabel('Temperatura (°C)')
ax.set_zlabel('Oxígeno (mg/L)')

plt.colorbar(sc)
ax.set_title('Relación entre Salinidad, Temperatura y Oxígeno con End-Members')
ax.legend()

plt.show()

In [ ]:
sig_errors = [0.0004, 0.02, 1.5, 0.005] 
n_WM=5 
M=1000 

In [ ]:
def generar_endmembers_y_errores(design_M, design_M_desvest, sig_errors):

    np.random.seed()
    design_M_values = design_M.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').values
    design_M_desvest_values = design_M_desvest.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').values

    endmembers = np.random.normal(design_M_values, design_M_desvest_values)
    matriz_endmembers = pd.DataFrame(endmembers, columns=design_M_desvest.columns[1:])
    sig_errors_random = np.random.normal(loc=sig_errors, scale=np.array(sig_errors) * 0.1)

    return matriz_endmembers, sig_errors_random

matriz_endmembers, sig_errors_random = generar_endmembers_y_errores(design_M, design_M_desvest, sig_errors)

In [ ]:
def estandarizar_matrices(matriz_endmembers, data):

    design_standarized = preprocessing.scale(matriz_endmembers, axis=1)
    design_standarization_check = pd.DataFrame()
    mean = pd.DataFrame(design_standarized).mean(axis=1)
    stdev = pd.DataFrame(design_standarized).std(axis=1)
    design_standarization_check['mean'] = mean
    design_standarization_check['stdev'] = stdev

    mass_vect = [1 for _ in range(design_standarized.shape[1])] 
    last_row = design_standarized.shape[0] 

    design_std = pd.DataFrame(design_standarized)
    design_std.loc[last_row, :] = mass_vect

    data_standarized = preprocessing.scale(data.dropna())

    data_std = pd.DataFrame(data_standarized)

    data_standarization_check = pd.DataFrame()
    data_mean = pd.DataFrame(data_standarized).mean(axis=0)
    data_stdev = pd.DataFrame(data_standarized).std(axis=0)
    data_standarization_check['mean'] = data_mean
    data_standarization_check['stdev'] = data_stdev

    return design_std, data_std, design_standarized, data_standarized, design_standarization_check, data_standarization_check

design_std, data_std, design_standarized, data_standarized, design_standarization_check, data_standarization_check = estandarizar_matrices(matriz_endmembers, data)

In [ ]:
def calcular_vector_ponderacion(design_M, n_WM, sig_errors, M):
   
    mean_sub = design_M.iloc[:,1:].sub(design_M.iloc[:,1:].mean(axis=1), axis=0)
    mean_sub_square = np.square(mean_sub)
    sigma_j = (mean_sub_square.sum(axis=1) / n_WM) ** 0.5
    W_vector = sigma_j ** 2 / sig_errors
    last_r = W_vector.shape[0]  
    W_vector.loc[last_r] = M 
    
    print("Vector de ponderación W con el término de balance de masas")
    print(W_vector)
    
    return W_vector

W_vector = calcular_vector_ponderacion(design_M, n_WM, sig_errors, M)

In [ ]:
def ponderar_matrices(design_std, data_std, W_vector, M):
  
    W_design_std = design_std.multiply(W_vector.values, axis=0)
    W_data_std = data_std.multiply(W_vector.iloc[:-1].values, axis=1)  
    mass_vect_data = [M for _ in range(data_std.shape[0])]
    last_col_data = W_data_std.shape[1]
    W_data_std.loc[:, last_col_data] = mass_vect_data

    return W_design_std, W_data_std

W_design_std, W_data_std = ponderar_matrices(design_std, data_std, W_vector, M)

In [ ]:
def calcular_ompa(W_design_std, W_data_std, design_columns):
    
    result = []
    residual = []
    err_percent = []

    for i in range(W_data_std.shape[0]):
        row_optim, res_optim = scipy.optimize.nnls(W_design_std, W_data_std.iloc[i,:]) 
        result.append(row_optim)
        residual.append(res_optim)
        err_percent.append(np.sum(row_optim) - 1)

    ompa_resultados = pd.DataFrame(result, columns=design_columns)
    ompa_resultados['error'] = err_percent

    return ompa

ompa_resultados = calcular_ompa(W_design_std, W_data_std, design_M.columns[1:].values)

print("Resultados del OMPA:")
print(ompa_resultados)

In [ ]:
def montecarlo_ompa(n_simulaciones, design_M, design_M_desvest, sig_errors, M, n_WM, data):

    tabla_resultados = []
    lista_w_vectors = []

    for sim in range(n_simulaciones):
        print(f"Simulación {sim+1}/{n_simulaciones}")
        
        metadatos = data_t[['Codi', 'Latitude', 'Longitude', 'Date', 'DEPTH', 'Bottom Depth']].reset_index(drop=True)
        
        matriz_endmembers, sig_errors_random = generar_endmembers_y_errores(design_M, design_M_desvest, sig_errors)
        
        design_std, data_std, _, _, _, _ = estandarizar_matrices(matriz_endmembers, data)
        
        W_vector = calcular_vector_ponderacion(design_M, n_WM, sig_errors_random, M)
        
        lista_w_vectors.append(W_vector.reset_index(drop=True))
        
        W_design_std, W_data_std = ponderar_matrices(design_std, data_std, W_vector, M)
        
        ompa_resultados = calcular_ompa(W_design_std, W_data_std, design_M.columns[1:].values)
        
        ompa_resultados = pd.concat([metadatos, ompa_resultados], axis=1)
        ompa_resultados.insert(0, "Simulacion", sim + 1)  
        print(ompa_resultados)
        
        tabla_resultados.append(ompa_resultados)

    return tabla_resultados, lista_w_vectors

n_simulaciones=1000

tabla_resultados_ompa, lista_w_vectors = montecarlo_ompa(n_simulaciones, design_M, design_M_desvest, sig_errors, M, n_WM, data)

In [ ]:
df = pd.concat(tabla_resultados_ompa, ignore_index=True)
df.to_csv("resultados_ompa_todas_simulaciones.csv", index=False)

In [ ]:
file_path = "resultados_ompa_todas_simulaciones.csv"
df = pd.read_csv(file_path)

variables = ['ENACW16', 'ENACW12', 'MW', 'LSW', 'NEADW', 'error']

stats_df = df.groupby('Codi')[variables].agg(['mean', 'median', 'std', 'min', 'max', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)])
stats_df.columns = ['_'.join(col).strip() for col in stats_df.columns]
stats_df.reset_index(inplace=True)

print(stats_df)

stats_df.to_csv("estadisticas_ompa.csv", index=False)

In [ ]:
ST = pd.read_csv("st_0")  
estadisticas = pd.read_csv("estadisticas_ompa.csv")  

columnas_necesarias = ["Codi", "Latitude", "Longitude", "DEPTH", "Bottom Depth"]  
ST = ST[columnas_necesarias]

df_final = estadisticas.merge(ST, on="Codi", how="left")  
df_final.to_csv("estadisticas_odv.csv", index=False)

print(df_final)

In [ ]:
estaciones = tabla_resultados_ompa[0]['Codi'].unique()
masas_agua = tabla_resultados_ompa[0].columns[7:]

for estacion in estaciones:

    datos_estacion = [df[df['Codi'] == estacion] for df in tabla_resultados_ompa]

    for masa in masas_agua:
        plt.figure(figsize=(8, 5))
        
        valores_masa = np.concatenate([df[masa].dropna().values for df in datos_estacion])
        if len(valores_masa) > 0:  
            sns.histplot(valores_masa, kde=True, bins=30, color="blue", edgecolor="black")
            
            plt.title(f"Distribución de {masa} en Estación {estacion}")
            plt.xlabel("Porcentaje")
            plt.ylabel("Frecuencia")
            plt.xlim(0, 1.5) 
            plt.grid(True)
            
            plt.show()

In [ ]:
df.columns = df.columns.str.strip()

variables = ['ENACW12', 'ENACW16','MW', 'LSW', 'NEADW', 'error']

plt.figure(figsize=(15, 10))

for i, var in enumerate(variables, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[var], kde=True, bins=30, color="skyblue")
    plt.title(f'Histograma de {var}')
    plt.xlabel('Porcentaje de masa')
    plt.ylabel('Frecuencia')
    plt.xlim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
df_w = pd.DataFrame(lista_w_vectors)

df_w.columns = ['Salinidad', 'Temperatura', 'Oxígeno', 'Spiciness', 'Balance_masa']

w_stats = df_w.describe().T  
print(w_stats)

In [ ]:
plt.figure(figsize=(12, 8))

variables = ['Salinidad', 'Temperatura', 'Oxígeno', 'Spiciness']
for i, var in enumerate(variables, 1):
    plt.subplot(2, 2, i)
    sns.histplot(df_w[var], bins=30, kde=True, color="skyblue")
    plt.title(f'Histograma de {var}')
    plt.xlabel('Valor')
    plt.ylabel('Frecuencia')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.violinplot(data=df_w.iloc[:, :-1])
plt.title("Violin Plot de las variables de ponderación")
plt.xlabel("Variables")
plt.ylabel("Valores de ponderación")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df_f = pd.read_csv("estadisticas_odv.csv")
df_grouped = df_f.groupby('DEPTH')[['ENACW16_mean', 'ENACW12_mean', 'MW_mean', 'LSW_mean', 'NEADW_mean']].mean()
print(df_grouped.head())

ax = df_grouped.plot(kind='bar', stacked=True, figsize=(12, 7)) 

ax.set_title('Proporciones de end-members por profundidad')
ax.set_xlabel('Profundidad (m)')
ax.set_ylabel('Proporción')
ax.legend(title='End-Members')
ax.set_ylim(0, 1)

n = 8  
ticks = ax.get_xticks()
labels = []
original_labels = [label.get_text() for label in ax.get_xticklabels()]

for i, tick_value in enumerate(ticks):
    if i % n == 0:
        try:
           
            label_text = original_labels[i] 
            int_label = int(float(label_text))
            labels.append(int_label)
        except (ValueError, IndexError):
            labels.append('')
    else:
        labels.append('')  
        
ax.set_xticks(ticks[::n])
ax.set_xticklabels(labels[::n], rotation=90) 

plt.tight_layout()
plt.show()